In [1]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import os

os.environ["PYSPARK_PYTHON"]="/home/pc/g5_env/bin/python39"

spark = SparkSession.builder.master("local[16]")\
            .appName("zh_pos_tag")\
            .config('spark.executor.memory', '30g')\
            .config('spark.driver.maxResultSize', '15g')\
            .config('spark.driver.memory', '30g')\
            .config('spark.ui.showConsoleProgress', False)\
            .getOrCreate()

sparkContext = spark.sparkContext

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 22:40:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/07 22:40:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/07 22:40:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/07 22:40:19 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
file_path = 'hdfs://g5.bigtop.it:8020/user/root/filtered_zh_wiki.parquet/'

In [3]:
df1 = spark.read.option("header",True).parquet(file_path)
df1.show()

+------------------------------+----------+----------+-------------------------------+--------------------------------+---------------------------------+--------------------------------+
|                token_sentence|sent_count|word_count|                         2grams|                          3grams|                           4grams|                          5grams|
+------------------------------+----------+----------+-------------------------------+--------------------------------+---------------------------------+--------------------------------+
|  [埃林頓, 是, 一個, 位, 於...|         1|        10|       [於 美國, 美國 康乃狄克]|  [位 於 美國, 於 美國 康乃狄...|    [一個 位 於 美國, 位 於 美...|   [是 一個 位 於 美國, 一個 ...|
|  [红宝石, 街道, 是, 中国, ...|         1|        16|       [是 中国, 中国 黑龙江省]|  [街道 是 中国, 是 中国 黑龙...|   [红宝石 街道 是 中国, 街道 ...|[红宝石 街道 是 中国 黑龙江省...|
|  [中國, 古代, 火藥, 火器, ...|         3|         7|           [史 研究, 研究 第頁]|    [火器 史 研究, 史 研究 第頁]|   [火藥 火器 史 研究, 火器 史...|  [古代 火藥 火器 史 研究, 火...|
|  [最終, 法國, 於, 決賽

In [4]:
from pyspark.sql.functions import col

gram2 = df1.select(col("2grams")).withColumnRenamed("2grams","n-grams")
gram3 = df1.select(col("3grams")).filter(df1.word_count > 2).withColumnRenamed("3grams","n-grams")
gram4 = df1.select(col("4grams")).filter(df1.word_count > 3).withColumnRenamed("4grams","n-grams")
gram5 = df1.select(col("5grams")).filter(df1.word_count > 4).withColumnRenamed("5grams","n-grams")

In [5]:
n_gram = gram5.union(gram4).union(gram3).union(gram2).persist(StorageLevel.MEMORY_ONLY)

In [6]:
n_gram.show()

+--------------------------------+
|                         n-grams|
+--------------------------------+
|   [是 一個 位 於 美國, 一個 ...|
|[红宝石 街道 是 中国 黑龙江省...|
|  [古代 火藥 火器 史 研究, 火...|
| [自年 奪冠後再 一次 在 世界,...|
|  [他 自幼 修习 中国 水墨, 自...|
|  [汉族 陕西 西安 人 中国, 陕...|
| [来 昆明 做龙云 的 统战工作,...|
| [法裔 的 美國 作曲家 瓦雷茲,...|
|[年 赴美 加利福尼亚 理工学院 ...|
|      [世界 体育 奖由 全球 组织]|
|  [彼得 畢夏帶 回 現實 世界, ...|
| [年月日 美國 軍隊 透過 城鎮,...|
|    [后若望 开始 担任 神职 工作]|
|      [一份 即興 劇場 演員 工作]|
|   [貝納 是 位 於 美國, 是 位...|
|[奈特 阿奇 博爾德前 美國 籃球...|
|  [上 唯一 同时 保持 世界, 唯...|
| [年月日 動工 經過 全體 工作,...|
|   [鎮區 是 位 於 美國, 是 位...|
|   [修建 时 采用 了 中国, 时 ...|
+--------------------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import explode
ngrams = n_gram.select(explode(col('n-grams'))).withColumnRenamed("col","ngrams")

In [8]:
ngrams.show(truncate=False)

+----------------------------------------+
|ngrams                                  |
+----------------------------------------+
|是 一個 位 於 美國                      |
|一個 位 於 美國 康乃狄克                |
|位 於 美國 康乃狄克 州托蘭縣            |
|於 美國 康乃狄克 州托蘭縣 的            |
|美國 康乃狄克 州托蘭縣 的 城鎮          |
|红宝石 街道 是 中国 黑龙江省            |
|街道 是 中国 黑龙江省 齐齐哈尔市        |
|是 中国 黑龙江省 齐齐哈尔市 富拉尔基区  |
|中国 黑龙江省 齐齐哈尔市 富拉尔基区 下辖|
|古代 火藥 火器 史 研究                  |
|火藥 火器 史 研究 第頁                  |
|自年 奪冠後再 一次 在 世界              |
|奪冠後再 一次 在 世界 盃                |
|一次 在 世界 盃 捧杯                    |
|他 自幼 修习 中国 水墨                  |
|自幼 修习 中国 水墨 及                  |
|修习 中国 水墨 及 西方                  |
|中国 水墨 及 西方 油画                  |
|汉族 陕西 西安 人 中国                  |
|陕西 西安 人 中国 大陆                  |
+----------------------------------------+
only showing top 20 rows



In [9]:
def f(x): return x   
def exchangePosition(x, y):
    return y, x

In [10]:
ngrams2 = ngrams.rdd.map(lambda x: (1,x))\
.flatMapValues(f)\
.map(lambda x: exchangePosition(x[0], x[1]))\
.reduceByKey(lambda a,b: a+b)\
.persist(StorageLevel.MEMORY_ONLY)

In [11]:
ngrams2.count()

6739157

In [12]:
column = ['ngram', 'gram_count']
ngrams2 = ngrams2.toDF(column)

22/10/07 22:40:42 WARN BlockManager: Task 144 already completed, not releasing lock for rdd_42_0


In [13]:
ngrams2.show()

+----------------------------------+----------+
|                             ngram|gram_count|
+----------------------------------+----------+
|          世界 体育 奖由 全球 组织|         1|
|        較佳年 畢業 於 美國 夏威夷|         1|
|    代表 中国 出戰 巴西 里约热内卢|        30|
|          中国 大陆 主流 媒体 禁播|         1|
|            中国 名人 榜 名列 第名|         6|
|      日 中央研究院 法律 學 研究所|         1|
|  中国 十五届 金鹰奖 优秀奖 电视剧|         1|
|    中国科学院 国家 天文台 主办 的|         1|
|              的 工作 卻 因為 華人|         1|
|          戰國 無雙 參加 編劇 工作|         1|
|              及 紐約 時報 工作 他|         1|
|            毕业 后 参加 工作 历任|         6|
|            迅速 处理 中国 事变 的|         1|
|        黄素 石楼 位于 中国 福建省|         1|
|              天主 敎 敎 區的 工作|         1|
|          江西 广丰 人 中国 计算机|         1|
|中国 第九个 五年计划 国家 科技攻关|         1|
|      开始 接受 模特儿 工作 並發現|         1|
|      里弗 賽德為 美國 職棒 大聯盟|         1|
|              泣 是 中国 古代 星官|         1|
+----------------------------------+----------+
only showing top 20 rows



In [14]:
ngram2 = ngrams2.drop('gram_count')

In [15]:
ngram2.show()

+----------------------------------+
|                             ngram|
+----------------------------------+
|          世界 体育 奖由 全球 组织|
|        較佳年 畢業 於 美國 夏威夷|
|    代表 中国 出戰 巴西 里约热内卢|
|          中国 大陆 主流 媒体 禁播|
|            中国 名人 榜 名列 第名|
|      日 中央研究院 法律 學 研究所|
|  中国 十五届 金鹰奖 优秀奖 电视剧|
|    中国科学院 国家 天文台 主办 的|
|              的 工作 卻 因為 華人|
|          戰國 無雙 參加 編劇 工作|
|              及 紐約 時報 工作 他|
|            毕业 后 参加 工作 历任|
|            迅速 处理 中国 事变 的|
|        黄素 石楼 位于 中国 福建省|
|              天主 敎 敎 區的 工作|
|          江西 广丰 人 中国 计算机|
|中国 第九个 五年计划 国家 科技攻关|
|      开始 接受 模特儿 工作 並發現|
|      里弗 賽德為 美國 職棒 大聯盟|
|              泣 是 中国 古代 星官|
+----------------------------------+
only showing top 20 rows



In [16]:
import pyspark.sql.functions as f
ngram2 = ngram2.withColumn('word_count', f.size(f.split(f.col('ngram'), ' '))).persist(StorageLevel.MEMORY_ONLY)
ngram2.show(10, False)

+--------------------------------+----------+
|ngram                           |word_count|
+--------------------------------+----------+
|世界 体育 奖由 全球 组织        |5         |
|較佳年 畢業 於 美國 夏威夷      |5         |
|代表 中国 出戰 巴西 里约热内卢  |5         |
|中国 大陆 主流 媒体 禁播        |5         |
|中国 名人 榜 名列 第名          |5         |
|日 中央研究院 法律 學 研究所    |5         |
|中国 十五届 金鹰奖 优秀奖 电视剧|5         |
|中国科学院 国家 天文台 主办 的  |5         |
|的 工作 卻 因為 華人            |5         |
|戰國 無雙 參加 編劇 工作        |5         |
+--------------------------------+----------+
only showing top 10 rows



In [17]:
ngram2.count()

6739157

In [18]:
keywords =  ['世界', '工作', '研究',' 人口', '美國',  '中国']

def keyword_position(text, n_gram):

    keyword = []
    index_of_keyword =[]
    
    tempList = list(text.split(" "))
    
    for x in keywords:
        i = 0
        for y in tempList:
            if x == y:
                keyword.append(x)
                index_of_keyword.append(i)
            i = i + 1
    
    if keyword == []:
        return
    
    if n_gram == 3 or n_gram == 2:
        return text
    
    if n_gram == 5:
        if 2 in index_of_keyword:
            return text
    
    if n_gram == 4:
        if 1 in index_of_keyword or 2 in index_of_keyword:
            return text

    return

In [19]:
from pyspark.sql import Row

row = Row("ngram")
ngram3 = ngram2.rdd.map(lambda x: (keyword_position(x[0], x[1]))).map(row).toDF().dropna(how='any').persist(StorageLevel.MEMORY_ONLY)

In [20]:
ngram3.count()

2862323

In [21]:
def zh_pos_tag(text):    
    import logging
    import jieba
    import jieba.posseg
        
    jieba.setLogLevel(logging.WARNING)
    text = text.replace(' ','')
        
    results= jieba.posseg.lcut(text)
    empList = []
    for x, y in results:
        empList.append(y)
    
    return empList

In [22]:
ngram3.rdd.getNumPartitions()

64

In [23]:
ngram3 = ngram3.repartition(128)

In [24]:
ngram3.count()

2862323

In [25]:
pos_tag = ngram3.rdd.map(lambda x: (x[0], zh_pos_tag(x[0]))).persist(StorageLevel.MEMORY_ONLY)

In [26]:
pos_tag.count()

2862323

In [27]:
columns = ['sentence', 'pos_tag']

df_pos_tag = pos_tag.toDF(columns)

22/10/07 22:54:25 WARN BlockManager: Task 665 already completed, not releasing lock for rdd_118_0


In [67]:
df_pos_tag.show(truncate = False)

+-----------------------+-----------------------------+
|sentence               |pos_tag                      |
+-----------------------+-----------------------------+
|滙 豐銀行 工作 之後他  |[yg, nr, ng, v, vn, u, nr, r]|
|东方文化 与 世界       |[nz, p, n]                   |
|由 美國 返港           |[p, ns, v, j]                |
|为 制定 中国           |[p, v, ns]                   |
|阿富汗 中国 新疆 和    |[nr, ns, ns, c]              |
|徐健 焜 中国           |[nr, v, zg, ns]              |
|成为 世纪 中国 首条    |[v, n, ns, m]                |
|的 起草 工作 中        |[uj, v, vn, f]               |
|曾 出任 中国           |[d, v, ns]                   |
|急遽 轉變 工作         |[a, v, vn]                   |
|中国 唱片 上海         |[ns, n, ns]                  |
|在 杭州 工作 期间 对   |[p, ns, vn, f, p]            |
|财富 中国 强 企业      |[n, ns, a, n]                |
|在 現實 世界 與 昴     |[p, n, n, p, g]              |
|纪凯林 岁 中国 经济    |[nr, m, ns, n]               |
|意识 中国 特色         |[n, ns, n]                   |
|年月日 中国 第一家 村镇|[t, ns, m, n]              

In [79]:
from pyspark.sql.functions import col, concat_ws
df_final = df_pos_tag.withColumn('pos_tag', concat_ws(', ', col('pos_tag')))

In [80]:
df_final.show()

+-----------------------+--------------------+
|               sentence|             pos_tag|
+-----------------------+--------------------+
|  滙 豐銀行 工作 之後他|yg, nr, ng, v, vn...|
|       东方文化 与 世界|            nz, p, n|
|           由 美國 返港|         p, ns, v, j|
|           为 制定 中国|            p, v, ns|
|    阿富汗 中国 新疆 和|       nr, ns, ns, c|
|           徐健 焜 中国|       nr, v, zg, ns|
|    成为 世纪 中国 首条|         v, n, ns, m|
|        的 起草 工作 中|        uj, v, vn, f|
|           曾 出任 中国|            d, v, ns|
|         急遽 轉變 工作|            a, v, vn|
|         中国 唱片 上海|           ns, n, ns|
|   在 杭州 工作 期间 对|     p, ns, vn, f, p|
|      财富 中国 强 企业|         n, ns, a, n|
|     在 現實 世界 與 昴|       p, n, n, p, g|
|    纪凯林 岁 中国 经济|        nr, m, ns, n|
|         意识 中国 特色|            n, ns, n|
|年月日 中国 第一家 村镇|         t, ns, m, n|
|      巴县 人 中国 政治|        ns, n, ns, n|
|           學性 別 研究|            n, d, vn|
|         中国 人物 全国|            ns, n, n|
+-----------------------+--------------------+
only s

In [81]:
import pyspark.sql.functions as f
df_final = df_final.withColumn('n-gram', f.size(f.split(f.col('pos_tag'), ' ')))

In [82]:
df_final.show()

+-----------------------+--------------------+------+
|               sentence|             pos_tag|n-gram|
+-----------------------+--------------------+------+
|  滙 豐銀行 工作 之後他|yg, nr, ng, v, vn...|     8|
|       东方文化 与 世界|            nz, p, n|     3|
|           由 美國 返港|         p, ns, v, j|     4|
|           为 制定 中国|            p, v, ns|     3|
|    阿富汗 中国 新疆 和|       nr, ns, ns, c|     4|
|           徐健 焜 中国|       nr, v, zg, ns|     4|
|    成为 世纪 中国 首条|         v, n, ns, m|     4|
|        的 起草 工作 中|        uj, v, vn, f|     4|
|           曾 出任 中国|            d, v, ns|     3|
|         急遽 轉變 工作|            a, v, vn|     3|
|         中国 唱片 上海|           ns, n, ns|     3|
|   在 杭州 工作 期间 对|     p, ns, vn, f, p|     5|
|      财富 中国 强 企业|         n, ns, a, n|     4|
|     在 現實 世界 與 昴|       p, n, n, p, g|     5|
|    纪凯林 岁 中国 经济|        nr, m, ns, n|     4|
|         意识 中国 特色|            n, ns, n|     3|
|年月日 中国 第一家 村镇|         t, ns, m, n|     4|
|      巴县 人 中国 政治|        ns, n, ns,

In [83]:
df_final = df_final.filter(col('n-gram') < 6)

In [84]:
df_final.count()

2745910

In [85]:
df_final.show()

+-----------------------+---------------+------+
|               sentence|        pos_tag|n-gram|
+-----------------------+---------------+------+
|       东方文化 与 世界|       nz, p, n|     3|
|           由 美國 返港|    p, ns, v, j|     4|
|           为 制定 中国|       p, v, ns|     3|
|    阿富汗 中国 新疆 和|  nr, ns, ns, c|     4|
|           徐健 焜 中国|  nr, v, zg, ns|     4|
|    成为 世纪 中国 首条|    v, n, ns, m|     4|
|        的 起草 工作 中|   uj, v, vn, f|     4|
|           曾 出任 中国|       d, v, ns|     3|
|         急遽 轉變 工作|       a, v, vn|     3|
|         中国 唱片 上海|      ns, n, ns|     3|
|   在 杭州 工作 期间 对|p, ns, vn, f, p|     5|
|      财富 中国 强 企业|    n, ns, a, n|     4|
|     在 現實 世界 與 昴|  p, n, n, p, g|     5|
|    纪凯林 岁 中国 经济|   nr, m, ns, n|     4|
|         意识 中国 特色|       n, ns, n|     3|
|年月日 中国 第一家 村镇|    t, ns, m, n|     4|
|      巴县 人 中国 政治|   ns, n, ns, n|     4|
|           學性 別 研究|       n, d, vn|     3|
|         中国 人物 全国|       ns, n, n|     3|
|         委员会 向 美國|       n, p, ns|     3|
+---